In [ ]:
import matplotlib as mpl

import pandas as pd
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
column_names = ["Tag", "TrainTime", "ReadTime", "IndexSize"]

# Read the data
df = pd.read_csv("data.dat", names=column_names)

# Split the 'tag' column into 'Method' and 'Dataset'
df[['Learned Index', 'Dataset', 'Workload', 'Range']] = df['Tag'].str.split('_', expand=True)

mpl.rcParams['font.family'] = 'Times New Roman'

num_datasets = df['Range'].nunique()
df['ReadTime'] = df['ReadTime'] / 10e7

markers = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', 'H']


wisckey_values = df[df['Learned Index'] == 'WiscKey'].groupby('Range')['ReadTime'].mean()
df['ReadTime'] = df.groupby('Range')['ReadTime'].transform(lambda x: x / wisckey_values[x.name])


# palette = sns.color_palette(['#e76254', '#ef8a47', '#f7aa58', '#ffd06f', '#ffc896', '#aadce0', '#72bcd5', '#528fad', '#376795', '#1e466e'])
palette = sns.color_palette()
plt.figure(figsize=(10, 6))
# sns.lineplot(data=df, x='Range', y='ReadTime', hue='Learned Index', linewidth=3)
learned_indexes = df['Learned Index'].unique()
# Create a line plot for each 'Learned Index'
for i, learned_index in enumerate(learned_indexes):
    df_subset = df[df['Learned Index'] == learned_index]
    sns.lineplot(data=df_subset, x='Range', y='ReadTime', marker=markers[i], markersize=20, linewidth=2.5, label=learned_index,color=palette[i], alpha=0.8)

# palette = sns.color_palette('muted')


plt.grid(True)

plt.tick_params(axis='both', labelsize=25)  # Set tick font size
plt.xlabel('Range', fontsize=30)  # Set x label font size
plt.ylabel('Relative Query Time Ratio', fontsize=30)  # Set y label font size
# plt.legend(title='Learned Index', title_fontsize='15', fontsize='14',bbox_to_anchor=(1, 1.02), loc='upper left')  # Set legend font size


# legend = plt.legend(title='Learned Index', title_fontsize='15', fontsize='14', bbox_to_anchor=(1.01, 0, 0.2, 3), loc='lower left', mode='expand', borderaxespad=0.)

legend = plt.legend( title_fontsize='15', fontsize='14', loc='upper center', bbox_to_anchor=(0.5, 1.2),
           fancybox=False, shadow=False, ncol=5)


plt.savefig("RangeQuery.pdf", format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
column_names = ["Tag", "TrainTime", "ReadTime", "IndexSize"]

# Read the data
df = pd.read_csv("data.dat", names=column_names)

# Split the 'tag' column into 'Method' and 'Dataset'
df[['Learned Index', 'Dataset', 'Workload', 'Range']] = df['Tag'].str.split('_', expand=True)

mpl.rcParams['font.family'] = 'Times New Roman'
palette = sns.color_palette()
plt.figure(figsize=(10, 6))

# 假设df是你的DataFrame，并且已经包含了需要的数据和列

# 对每个Learned Index进行平滑处理并绘图
markers = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', 'H']
wisckey_values = df[df['Learned Index'] == 'WiscKey'].groupby('Range')['ReadTime'].mean()
df['ReadTime'] = df.groupby('Range')['ReadTime'].transform(lambda x: x / wisckey_values[x.name])
learned_indexes = df['Learned Index'].unique()

for i, learned_index in enumerate(learned_indexes):
    df_subset = df[df['Learned Index'] == learned_index]
    
    # 将'Range'列转换为数字，用于插值
    x = df_subset['Range'].astype(np.float)
    y = df_subset['ReadTime'].astype(np.float)
    
    # 生成平滑曲线的新x值
    x_new = np.linspace(x.min(), x.max(),5)
    
    # 使用make_interp_spline创建一个平滑的曲线
    spl = make_interp_spline(x, y, k=1)  # k是平滑曲线的程度
    y_smooth = spl(x_new)
    
    # 绘制平滑后的曲线
    plt.plot(x_new, y_smooth, marker=markers[i], markersize=8, linewidth=2, label=learned_index, color=palette[i])

plt.grid(True)
plt.tick_params(axis='both', labelsize=25)
plt.xlabel('Range', fontsize=30)
plt.ylabel('Relative Query Time Ratio', fontsize=30)

legend = plt.legend(title='Learned Index', title_fontsize='15', fontsize='14', bbox_to_anchor=(1.01, 0, 0.2, 3), loc='lower left', mode='expand', borderaxespad=0.)
plt.savefig("RangeQuery_smooth.pdf", format='pdf', bbox_inches='tight')
plt.show()